In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/My\ paper

/content/drive/Shareddrives/My paper


In [ ]:
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime

In [ ]:
member = pd.read_csv('dataset/91APP_MemberData.csv')

In [ ]:
order = pd.read_csv('dataset/91APP_OrderData.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
print(len(member['MemberID'].unique()), len(order['MemberID'].unique()))

377133 235493


In [ ]:
order['TradesDateTime'] = pd.to_datetime(order['TradesDateTime'])
order = order.sort_values(by='TradesDateTime')

In [ ]:
member['RegisterDateTime'] = pd.to_datetime(member['RegisterDateTime'])
member = member.sort_values(by='RegisterDateTime')
member = member.drop_duplicates(subset=['MemberID'], keep=False)

In [ ]:
len(member)

377132

In [ ]:
label_data = pd.read_csv('./Customer Value Prediction/TABLES/Label Data.csv')

In [ ]:
memberIDs = list(label_data['MemberID'].unique())

## 加上 UID 欄位


In [ ]:
def add_uid_col(data):
  df = data.copy()
  df['uid'] = df['MemberID'].str.replace('+', '%2B')
  df['uid'] = df['uid'].str.replace('/', '%2F')
  df['uid'] = df['uid'].str.replace('=', '%3D')

  return df['uid']

In [ ]:
label_data['uid'] = add_uid_col(label_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
uids = list(label_data['uid'].unique())

In [ ]:
print(len(memberIDs), len(uids))

83931 83931


## 讀取所有行為資料

In [ ]:
behaviors = pd.read_csv('all behaviors data.csv')

In [ ]:
behaviors.columns

Index(['Unnamed: 0', 'dc', 'did', 'uid', 'DateTime', 'bh', 'uid_filled'], dtype='object')

In [ ]:
behaviors = behaviors[behaviors['uid_filled'].isin(uids)][['did', 'uid', 'DateTime', 'bh', 'uid_filled']]

In [ ]:
len(behaviors)

12879698

In [ ]:
behaviors = behaviors.sort_values(by=['uid_filled', 'DateTime']).groupby(by=['uid_filled']).tail(60)

In [ ]:
def calc_time(df, current_time):
  df_copy = df.copy()

  df_copy['DateTime'] = pd.to_datetime(df_copy['DateTime'], format='%Y-%m-%d %H:%M:%S')
  df_copy['datetime_next'] = df_copy['DateTime'].shift(periods=-1, fill_value=current_time)

  df_copy['uid_next'] = df_copy['uid_filled'].shift(periods=-1, fill_value='final_member')
  df_copy.loc[(df_copy['uid_filled'] != df_copy['uid_next']), 'datetime_next'] = current_time

  # 最後一個行為的 TimeDiff = 該行為到當今時間的時間差
  df_copy['TimeDiff'] = (df_copy['datetime_next'] - df_copy['DateTime']).astype('timedelta64[s]')

  return df_copy[['DateTime', 'bh', 'uid_filled', 'TimeDiff']]

In [ ]:
bh_df = calc_time(behaviors, '2020-04-30 23:59:59.999')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Passing <class 'str'> to shift is deprecated and will raise in a future version, pass Timestamp instead.
  """


In [ ]:
bh_df.drop_duplicates(subset=['uid_filled'], keep='last')

,DateTime,bh,uid_filled,TimeDiff
311,2019-12-16 05:59:12.966,viewmainpage,%2B%2B%2FsALKlNdkh585nb%2F9geIrN84FIZZLkD8qf90...,11815247.0
401,2019-11-24 11:27:10.417,viewmainpage,%2B%2B3XVjBiQPLoAXWc7gK%2BTdvTH4J4AmgkeFlB3mmN...,13696369.0
776,2020-04-11 08:40:11.394,viewproduct,%2B%2B44iA8rzzl4lYN3dzParPLqAXetHBSwDFIBUBD%2B...,1696788.0
780,2019-07-18 12:04:25.985,viewcategory,%2B%2B6Xs%2FwHblxbDDDYQ9osx6DMUooacHVpAJvv7KWg...,24839734.0
888,2020-02-17 00:37:35.489,viewmainpage,%2B%2B8lPULC%2BdY%2F3MWf4kbymzr1cx%2B3Eaktr2fv...,6391344.0
...,...,...,...,...
15356608,2019-12-08 11:45:38.296,viewproduct,zzrOAdwnKzCrEW7aUK%2FMkdXLVyQeo59HdxcfqMqwiwA%3D,12485661.0
15356645,2020-04-17 04:21:05.275,viewmainpage,zzvTsLQAKUueMvz%2Bvx%2FYptKHZIMMEu7nSWnkOiBhtR...,1193934.0
15356812,2019-12-23 14:44:31.693,viewmainpage,zzw%2Bk3PFtOtuOIu62cuXNkLNLUFDlgmXZj2XeMTwl2g%3D,11178928.0
15357012,2020-01-04 09:23:50.628,traffic,zzw9O6yOUvVbxgrvUGCnmjjacARx%2FbQ%2F12aZDz27Tb...,10161369.0


In [ ]:
pd.merge(label_data, bh_df, left_on='uid', right_on='uid_filled', how='right').drop(['uid', 'uid_filled', 'DateTime'], axis=1).to_csv('Event Sequence base on second.csv', index=False)

In [ ]:
bh_df['bh_codes'] = pd.Categorical(bh_df.bh).codes
bh_df['bh_codes'] = bh_df['bh_codes'] + 1 # 因為之後會補 0 

In [ ]:
bhs = pd.DataFrame(bh_df.groupby('uid_filled')['bh'].apply(list))
bh_codes= pd.DataFrame(bh_df.groupby('uid_filled')['bh_codes'].apply(list))
TimeDiffs = pd.DataFrame(bh_df.groupby('uid_filled')['TimeDiff'].apply(list))
Lens = pd.DataFrame(bh_df.groupby('uid_filled').size(), columns=['Events'])

In [ ]:
result = pd.merge(bhs, bh_codes, on='uid_filled')
result = pd.merge(result, TimeDiffs, on='uid_filled')
result = pd.merge(result, Lens, on='uid_filled')

In [ ]:
len(result)

83931

In [ ]:
result.describe()

,Events
count,83931.000000
mean,41.102620
std,22.297833
min,1.000000
25%,18.000000
50%,59.000000
75%,60.000000
max,60.000000


In [ ]:
result.columns

Index(['bh', 'bh_codes', 'TimeDiff', 'Events'], dtype='object')

In [ ]:
result = pd.merge(label_data[['MemberID', 'uid']], result, left_on='uid', right_on='uid_filled', how='right').drop(['uid'], axis=1)

In [ ]:
len(result)

83931

In [ ]:
result = pd.merge(result, label_data[['MemberID', 'TotalSalesAmount', 'VIP']], on='MemberID', how='right')

In [ ]:
result.to_csv('Session Event Sequence.csv', index=False)

In [ ]:
result.head()

,MemberID,bh,bh_codes,TimeDiff,Events,TotalSalesAmount,VIP
0,RSQpIPxqSK2ooLPuUwWw8HhTN73mFGsmGPEmdarbdGM=,"[viewmainpage, viewmainpage, viewmainpage, vie...","[9, 9, 9, 9, 9, 9, 8, 9, 8, 9, 8, 8, 8, 8, 8, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",32,0.0,0.0
1,AqBrlkHltks4neA3Fmtq+yfiI4F7aZGODqKPeF3zGsA=,"[viewcategory, viewproduct, viewproduct, viewp...","[8, 10, 10, 10, 10, 10, 9, 9, 9, 8, 8, 10, 10,...","[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60,0.0,0.0
2,x/e5hrkgQCnCkhBzvk8Hs8i5dnX4tHRlrjHtKm6yvLg=,"[traffic, viewcategory, traffic, traffic, traf...","[6, 8, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",38,0.0,0.0
3,r/d/6e+NmTz+PFg45srs0HNMcUh1sWlRP27kZifm69I=,"[viewmainpage, viewmainpage, viewmainpage, vie...","[9, 9, 9, 9, 9, 9, 9, 9, 10, 9, 9, 9, 8, 8, 9,...","[0.0, 123.0, 0.0, 0.0, 79.0, 1.0, 1101.0, 0.0,...",60,7612.0,1.0
4,wouxhlfcOpZcJH6lJhyuQ+4lV8b7X0foxY7YQiOQwC4=,"[viewmainpage, viewmainpage, viewactivity, vie...","[9, 9, 7, 9, 8, 8, 9, 8, 9, 8, 8, 9, 8, 9, 8, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",56,0.0,0.0
